**Agradecimientos:** El contenido del siguiente jupyter notebook es la traducción al español del jupyter dispuesto en el siguiente [repositorio](https://github.com/wiqaaas/youtube/tree/master/Machine_Learning_from_Scratch/Search_Engine_Recommender) desarrollado por **Waqas Ahmed**.
Aquellas partes resaltadas en *cursivas* son aportes del profesor **Juan C. Correa**

# PageRank
Este jupyter supone que usted tiene conocimiento de autovectores y autovalores (*pero acá obviaremos eso por el momento*) y acá se muestra cómo se aplican con el algoritmo PageRank. Este jupyter consta de dos partes. La primera es una hoja de trabajo para que se ponga al día con el funcionamiento del algoritmo; aquí veremos un micro Internet con menos de 10 sitios web y veremos qué hace y qué puede salir mal. La segunda es una evaluación que pondrá a prueba su aplicación de la teoría propia a este problema escribiendo código y calculando el rango de página de una gran red que representa una subsección de Internet.

### 1- Page Rank en una red de Internet ficticia
##### Introducción

PageRank (desarrollado por Larry Page y Sergey Brin) revolucionó la búsqueda web al generar una lista clasificada de páginas web basada en la conectividad subyacente de la web. 

El algoritmo de PageRank se basa en un internauta aleatorio ideal que, al llegar a una página, pasa a la página siguiente haciendo clic en un enlace. 

El internauta tiene la misma probabilidad de hacer clic en cualquier enlace de la página y, cuando llega a una página sin enlaces, tiene la misma probabilidad de moverse a cualquier otra página escribiendo su URL. 

Además, el internauta puede optar ocasionalmente por escribir una URL aleatoria en lugar de seguir los enlaces de una página. 

El PageRank es el orden de clasificación de las páginas desde la página más probable hasta la menos probable que verá el internauta.


In [ ]:
%pylab notebook
import numpy as np
import numpy.linalg as la
np.set_printoptions(suppress=True)

### PageRank como un problema de algebra lineal

Imaginemos una pequeña red de Internet con apenas 6 páginas web, como las siguientes (**A**vocado, **B**ullseye, **C**atBabel, **D**romeda, **e**Tings, y **F**aceSpace). Cada página se vincula con algunas otras, y esta serie de conexiones forma una red como la siguiente

![A Micro-Internet](internet.png "A Micro-Internet")

El principio de diseño de PageRank es que los sitios web importantes estarán vinculados por sitios web importantes (*algo relacionado en teoría de redes con el concepto de Homofilia*). Este principio algo recursivo formará la base de nuestro pensamiento.

Imagina que tenemos 100 personas procrastinando en una Internet pequeña y cada uno ve un solo sitio web a la vez. Cada minuto, las personas siguen un enlace en su sitio web a otro sitio en esta Internet pequeña. Después de un tiempo, los sitios web a los que están más vinculados tendrán más personas visitándolos y, a la larga, cada minuto por cada persona que abandone un sitio web, entrará otro manteniendo constante el número total de personas en cada sitio web. El PageRank es simplemente la clasificación de los sitios web por la cantidad de personas que tiene cada página al final de este proceso.

Representamos el número de Pats en cada sitio web con el vector,

$$\mathbf{r} = \begin{bmatrix} r_A \\ r_B \\ r_C \\ r_D \\ r_E \\ r_F \end{bmatrix}$$
Y decimos que el número de personas en cada página web en el minuto $i+1$ está relacionado a aquellos en el minuto $i$ por la matriz de transformación siguiente:

$$ \mathbf{r}^{(i+1)} = L \,\mathbf{r}^{(i)}$$
with the matrix $L$ taking the form,
$$ L = \begin{bmatrix}
L_{A→A} & L_{B→A} & L_{C→A} & L_{D→A} & L_{E→A} & L_{F→A} \\
L_{A→B} & L_{B→B} & L_{C→B} & L_{D→B} & L_{E→B} & L_{F→B} \\
L_{A→C} & L_{B→C} & L_{C→C} & L_{D→C} & L_{E→C} & L_{F→C} \\
L_{A→D} & L_{B→D} & L_{C→D} & L_{D→D} & L_{E→D} & L_{F→D} \\
L_{A→E} & L_{B→E} & L_{C→E} & L_{D→E} & L_{E→E} & L_{F→E} \\
L_{A→F} & L_{B→F} & L_{C→F} & L_{D→F} & L_{E→F} & L_{F→F} \\
\end{bmatrix}
$$
donde las columnas representan la probabilidad de dejar una página para ir a cualquier otra, y sumarle 1. Las filas determinan cuán probable es que usted entre a una página a partir de cualquier otra, aunque estas probabilidades no necesitan sumar uno. El comportamiento a largo plazo de este sistema es cuando se cumple la siguiente igualdad $ \mathbf{r}^{(i+1)} = \mathbf{r}^{(i)}$, así que al quitar los sub-indices de esta fórmula podemos escribir,
$$ L \,\mathbf{r} = \mathbf{r}$$

lo cual es una ecuación de autovalor para la matriz $L$, con autovalor 1 (esto está garantizado por la estructura probabilística de dicha matriz).

In [ ]:
L = np.array([[0,   1/2, 1/3, 0, 0,   0 ],
              [1/3, 0,   0,   0, 1/2, 0 ],
              [1/3, 1/2, 0,   1, 0,   1/2 ],
              [1/3, 0,   1/3, 0, 1/2, 1/2 ],
              [0,   0,   0,   0, 0,   0 ],
              [0,   0,   1/3, 0, 0,   0 ]])

En principio, podríamos usar una biblioteca de álgebra lineal, como se muestra a continuación, para calcular los valores propios y los vectores. Y esto funcionaría para un sistema pequeño. Pero esto se vuelve inmanejable para sistemas grandes (*aquí entra la magia, la fama, y la reputación de lo que se llama Big Data*). Y dado que solo nos importa el vector propio principal (el que tiene el valor propio más grande, que será 1 en este caso), podemos usar el **método de iteración de potencia** que escalará mejor y es más rápido para sistemas grandes.

Utilice el siguiente código para echar un vistazo al PageRank de este micro Internet. (*Ahora sí toca entender intuitivamente qué es un autovalor y un autovector mirando este [video](https://youtu.be/YtgU1ozMgS0)* )

In [ ]:
eVals, eVecs = la.eig(L) # Obtenga los autovalores y autovectores
order = np.absolute(eVals).argsort()[::-1] # Ordenelos por su autovalores
eVals = eVals[order]
eVecs = eVecs[:,order]

r = eVecs[:, 0] # Defina r como el autovector principal
100 * np.real(r / np.sum(r)) # Haga que este autovector sume 1, y luego multipliquelo por 100 (las 100 personas)

Podemos ver en esta lista, la cantidad de personas que esperamos encontrar en cada sitio web después de mucho tiempo. Poniéndolos en orden de popularidad (según esta métrica), el PageRank de este micro Internet es:

CatBabel, Dromeda, Aguacate, FaceSpace, Bullseye, eTings

Volviendo al diagrama de nuestra pequeña internet, ¿es esto lo que hubiera esperado? Convénzase usted mismo de que, según la importancia aparente de las páginas, en términos de qué otras páginas se enlazan con ellas, esta es una clasificación sensata.

Intentemos ahora obtener el mismo resultado usando el método de potencia de iteración que se cubrió en este [video](https://youtu.be/a5zPyhQf7xw). Este método será mucho mejor para trabajar con sistemas grandes.

Primero, configuremos nuestro vector inicial, $\mathbf{r}^{(0)}$, de modo que tengamos las 100 personas distribuidas equitativamente en cada uno de nuestros 6 sitios web.

In [ ]:
r = 100 * np.ones(6) / 6 # Sets up this vector (6 entries of 1/6 × 100 each)
r # Shows it's value

Ahora, vamos a actualizar el vector al siguiente minuto con la matriz $L$.
Y corremos nuevamente en un loop lo anterior para ver si se estabiliza.

In [ ]:
for i in np.arange(100) : # Repeat 100 times
    r = L @ r
r

O mejor aún, podemos mantener corriendo las sintaxis hasta que lleguemos a un punto tolerable.

In [ ]:
r = 100 * np.ones(6) / 6 # Sets up this vector (6 entries of 1/6 × 100 each)
lastR = r
r = L @ r
i = 0
while la.norm(lastR - r) > 0.01 :
    lastR = r
    r = L @ r
    i += 1
print(str(i) + " iterations to convergence.")
r

Observe cómo el orden de PageRank se estabiliza muy rápidamente, y el vector converge sobre el valor calculado previamente luego de unas pocas decenas de repeticiones.

***¡Genial parce! ¡Usted acaba de calcular su primer PageRank!***

#### Parámetro de Amortiguación

El sistema que acabamos de estudiar convergió con bastante rapidez a la respuesta correcta. Consideremos una extensión de nuestro micro Internet donde las cosas comienzan a ir mal.

Digamos que una nueva página se añade: *Geoff's* Website.
Esta website está vinculada a *FaceSpace* y solo se vincula a sí misma.
![An Expanded Micro-Internet](internet2.png "An Expanded Micro-Internet")

Intuitivamente, solo *FaceSpace*, que se encuentra en la mitad inferior de la clasificación de la página, enlaza con este sitio web entre los otros dos a los que enlaza, por lo que podríamos esperar que el sitio de *Geoff* tenga una puntuación de PageRank correspondientemente baja.

Construyamos la nueva matriz de $L$ para esta micro Internet expandida y usemos Power-Iteration en el vector de visitantes. Mira qué pasa…

In [ ]:
 # Llamaremos a esta matriz, L2, para distinguirla de la previa.
L2 = np.array([[0,   1/2, 1/3, 0, 0,   0, 0 ],
               [1/3, 0,   0,   0, 1/2, 0, 0 ],
               [1/3, 1/2, 0,   1, 0,   0, 0 ],
               [1/3, 0,   1/3, 0, 1/2, 0, 0 ],
               [0,   0,   0,   0, 0,   0, 0 ],
               [0,   0,   1/3, 0, 0,   1, 0 ],
               [0,   0,   0,   0, 0,   0, 1 ]])

In [ ]:
r = 100 * np.ones(7) / 7 # Fijemos este vector (7 entradas of 1/7 × 100 c/u)
lastR = r
r = L2 @ r
i = 0
while la.norm(lastR - r) > 0.01 :
    lastR = r
    r = L2 @ r
    i += 1
print(str(i) + " iterations to convergence.")

¡Nada bueno! *Geoff* parece estar tomando todo el tráfico en el micro-Internet y de alguna manera llegando a la cima del PageRank. Este comportamiento se puede entender, porque una vez que un visitante llega al sitio web de *Geoff*, no puede irse, ya que todos los enlaces regresan a Geoff.

Para combatir esto, podemos agregar una pequeña probabilidad de que los visitantes no sigan ningún enlace en una página web, sino que visiten un sitio web en el micro Internet al azar. Diremos que la probabilidad de que sigan un enlace es de $d$ y, por lo tanto, la probabilidad de elegir un sitio web aleatorio es de $1-d$. Podemos usar una nueva matriz para calcular dónde ocurre la visita cada minuto.

$$ M = d \, L + \ frac {1-d} {n} \, J $$

donde $J$ es una matriz $ n \ times n $ donde cada elemento es uno.

Si $d$ es uno, tenemos el caso que teníamos anteriormente, mientras que si $ d $ es cero, siempre visitaremos una página web aleatoria y, por lo tanto, todas las páginas web tendrán la misma probabilidad y la misma clasificación. Para que esta extensión funcione mejor, $ 1-d $ debería ser algo pequeño, aunque no entraremos en una discusión sobre exactamente qué tan pequeño.

Intentemos de nuevo este PageRank con esta extensión.

In [ ]:
d = 0.5 # Feel free to play with this parameter after running the code once.
M = d * L2 + (1-d)/7 * np.ones([7, 7]) # np.ones() is the J matrix, with ones for each entry.

In [ ]:
r = 100 * np.ones(7) / 7 # Sets up this vector (6 entries of 1/6 × 100 each)
lastR = r
r = M @ r
i = 0
while la.norm(lastR - r) > 0.01 :
    lastR = r
    r = M @ r
    i += 1
print(str(i) + " iterations to convergence.")
r

Sin duda, esto es mejor, el PageRank proporciona números razonables para los visitantes que terminan en cada página web. Sin embargo, este método todavía predice que *Geoff* tiene una página web de alto rango. Esto podría verse como una consecuencia del uso de una red pequeña. También podríamos solucionar el problema si no contamos los autoenlaces al producir la matriz $L$ (y si un sitio web no tiene enlaces salientes, conviértalo en todos los sitios web por igual). No buscaremos más en esta ruta, ya que se trata de mejorar en el PageRank, en vez de observar sus propios problemas.

Ahora está en una buena posición, después de haber adquirido un conocimiento de PageRank, para producir su propio código para calcular el PageRank de un sitio web con miles de entradas.

¡Buena suerte!

### 2. Sub Internet Page Ranking
En esta evaluación, usted se preguntará cómo producir una función que calcule el PageRank para una matriz con un número arbitrariamente grande de páginas.

In [ ]:
# Complete esta función para brindarle a PageRank la capacidad de trabajar con un número arbitrario de páginas.
# I.e. el autovector principal del sistema amortiguado, usando el método power iteration.
# (Acá la normalización no importa)
# Los argumentos de la función son los de la matriz, y d el parametro de amortiguación - como se definió arriba.
def pageRank(linkMatrix, d) :
    n = linkMatrix.shape[0]
    M = d * linkMatrix + (1-d)/n * np.ones([n, n])
    r = 100 * np.ones(n) / n # Sets up this vector (n entries of 1/n × 100 each)
    last = r
    r = M @ r
    while la.norm(last - r) > 0.01 :
        last = r
        r = M @ r
    return r

In [ ]:
def generate_internet(n) :
    c = np.full([n,n], np.arange(n))
    c = (abs(np.random.standard_cauchy([n,n])/2) > (np.abs(c - c.T) + 1)) + 0
    c = (c+1e-10) / np.sum((c+1e-10), axis=0)
    return c

In [ ]:
# Use la siguiente función para generar internets de distinto tamaño
generate_internet(5)

In [ ]:
# Pruebe su PageRank y compárelo con la función predefinida de autovalores "eig".
# Debería ver que el suyo es mucho más rápido para internets de mayor tamaño
L = generate_internet(100)

In [ ]:
pageRank(L, 1)

In [ ]:
# Observe que esto está calculando los autovalores de la matriz L,
# sin ninguna amortiguación. Puede dar resultados diferentes que su función pageRank.
# Si lo desea, usted podría modificar esta celda para incluir la amortiguación.
eVals, eVecs = la.eig(L) # Obtenga los autovalores y autovectores
order = np.absolute(eVals).argsort()[::-1] # Ordene por autovalores
eVals = eVals[order]
eVecs = eVecs[:,order]

r = eVecs[:, 0]
100 * np.real(r / np.sum(r))

In [ ]:
# Si usted quisiera ver gráficamente el pagerank
# Esta sintaxis dibuijará un gráfico de barras, para cada página enumerada del internet,
# La altura de cada barra será el puntaje del PageRank.
# Corra esta sintaxis para ver el PageRank para cada internet que usted genere.
# Con suerte, usted debería ver lo que esperaría
# - hay pocos clusters de páginas importantes, pero la mayoría de las páginas en Internet son bobadas o tonterías!
%pylab notebook
r = pageRank(generate_internet(100), 0.9)
plt.bar(arange(r.shape[0]), r);